In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd

# Read the text file into a DataFrame
df = pd.read_csv('thedataset.csv')

In [3]:
df.head()

,Eng,Beng
0,the,দ্য
1,and,এবং
2,have,আছে
3,that,যে
4,for,জন্য


In [4]:
english_sentences = df["Eng"].tolist()
bengali_sentences = df["Beng"].tolist()

In [5]:
tokenizer_eng = Tokenizer()
tokenizer_eng.fit_on_texts(english_sentences)
eng_seq = tokenizer_eng.texts_to_sequences(english_sentences)

tokenizer_beng = Tokenizer()
tokenizer_beng.fit_on_texts(bengali_sentences)
beng_seq = tokenizer_beng.texts_to_sequences(bengali_sentences)


vocab_size_eng = len(tokenizer_eng.word_index) + 1
vocab_size_beng = len(tokenizer_beng.word_index) + 1

# Padding
max_length = max(len(seq) for seq in eng_seq + beng_seq)
eng_seq_padded = pad_sequences(eng_seq, maxlen=max_length, padding='post')
beng_seq_padded = pad_sequences(beng_seq, maxlen=max_length, padding='post')

In [6]:
embedding_dim = 256
units = 512

# Encoder
encoder_inputs = Input(shape=(max_length,))
enc_emb = Embedding(input_dim=vocab_size_eng, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_length,))
dec_emb_layer = Embedding(input_dim=vocab_size_beng, output_dim=embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(vocab_size_beng, activation='softmax')
output = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], output)

# Compilation of model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [7]:
X_train, X_val, y_train, y_val = train_test_split(eng_seq_padded, beng_seq_padded, test_size=0.2)
model.fit([X_train, X_train], y_train, validation_data=([X_val, X_val], y_val), epochs=10, batch_size=64)

Epoch 1/10
129/129 [==============================] - 236s 2s/step - loss: 1.7313 - accuracy: 0.8376 - val_loss: 1.2483 - val_accuracy: 0.8486
Epoch 2/10
129/129 [==============================] - 202s 2s/step - loss: 1.1633 - accuracy: 0.8517 - val_loss: 1.1874 - val_accuracy: 0.8540
Epoch 3/10
129/129 [==============================] - 150s 1s/step - loss: 1.0989 - accuracy: 0.8539 - val_loss: 1.1647 - val_accuracy: 0.8543
Epoch 4/10
129/129 [==============================] - 137s 1s/step - loss: 1.0517 - accuracy: 0.8568 - val_loss: 1.1543 - val_accuracy: 0.8553
Epoch 5/10
129/129 [==============================] - 133s 1s/step - loss: 1.0119 - accuracy: 0.8580 - val_loss: 1.1347 - val_accuracy: 0.8562
Epoch 6/10
129/129 [==============================] - 134s 1s/step - loss: 0.9688 - accuracy: 0.8597 - val_loss: 1.1285 - val_accuracy: 0.8580
Epoch 7/10
129/129 [==============================] - 146s 1s/step - loss: 0.9257 - accuracy: 0.8618 - val_loss: 1.1152 - val_accuracy: 0.8599

In [8]:
def translate_sentence(sentence):
    seq = tokenizer_eng.texts_to_sequences([sentence])
    padded = pad_sequences(seq, maxlen=max_length, padding='post')
    translated = np.argmax(model.predict([padded, padded]), axis=-1)
    
    translated_sentence = []
    for i in translated[0]:
        if i in tokenizer_beng.index_word:
            translated_sentence.append(tokenizer_beng.index_word[i])
        else:
            translated_sentence.append(' ')  
        
    return ' '.join(translated_sentence)

In [14]:
input_sentence = input('Enter your sentence:')
translated_sentence = translate_sentence(input_sentence)
print(f"Input: {input_sentence}")
print(f"Translated: {translated_sentence}")

Enter your sentence:i hate you
1/1 [==============================] - 0s 32ms/step
Input: i hate you
Translated: আমি আমার আপনি করি।                                
